In [1]:
import numpy as np
import pandas as pd
from Bio import Phylo
from sklearn.cluster import AgglomerativeClustering
from scipy.spatial.distance import squareform, pdist, cdist
import random
from sklearn.neighbors import kneighbors_graph
import matplotlib.pyplot as plt
import seaborn as sns



def get_dist_matrix_from_tree(tree_file):
    """
    Performs phylogenetic block k-fold cross-validation.

    Args:
        tree_file (str): Path to the phylogenetic tree file (Newick format).
        fold_list (list): List of the number of folds to test.

    Returns:
        list: List of dictionaries, each containing fold assignments and distance metrics.
    """

    # Load phylogenetic tree
    tree = Phylo.read(tree_file, "newick")
    # Get tip names
    tip_names = [terminal.name for terminal in tree.get_terminals()]
    #dist_matrix = tree.distance_matrix().values

    #Create distance matrix from the tree
    dist_matrix = np.zeros((len(tree.get_terminals()), len(tree.get_terminals())))
    for i, terminal1 in enumerate(tree.get_terminals()):
        for j, terminal2 in enumerate(tree.get_terminals()):
            if i < j:  # Only calculate upper triangle to avoid redundancy
                dist_matrix[i, j] = tree.distance(terminal1, terminal2)
                dist_matrix[j, i] = dist_matrix[i, j]  # Mirror for symmetry
    
    #tips_for_tr = []
    #for names in tip_names:
    #    tips_for_tr.append(f'd_{names}')
    #print(tips_for_tr)
    dist_df =  pd.DataFrame(dist_matrix, index=tip_names, columns=tip_names)

            
    return dist_df, dist_matrix, tip_names


In [2]:
tree_file = "./opsin_wt_tree/vpod_1.1_wt/wt_aligned_VPOD_1.1_het.fasta.treefile"
dist_df, dist_matrix, tip_names = get_dist_matrix_from_tree(tree_file)

In [13]:
import numpy as np

def phylogenetic_clustering(distance_matrix, n_initial_bins, distance_threshold):
    """
    Clusters terminal leaves of a phylogenetic tree into bins based on distance.

    Args:
        distance_matrix: A square numpy array representing pairwise distances.
        n_initial_bins: The number of initial bins to create with the most distant points.
        distance_threshold: The minimum allowable distance for bin assignment.

    Returns:
        A numpy array of class assignments (bin numbers) for each terminal leaf.
    """

    n_leaves = distance_matrix.shape[0]
    class_assignments = np.full(n_leaves, -1, dtype=int)  # Initialize as -1 (unassigned)
    upper_triangle_indices = np.triu_indices_from(distance_matrix, k=1)
    print(f'This is the upper triangle:\n{upper_triangle_indices}')
    # 1. Initialize Bins with Most Distant Points:
    initial_points = np.unravel_index(np.argsort(distance_matrix[upper_triangle_indices], axis=None)[-n_initial_bins:], distance_matrix.shape)
    print(f'These are the intial points: {initial_points}')
    print(f'This is the length of the intial points: {len(initial_points)}')

    for i, (idx1, idx2) in enumerate(zip(*initial_points)):
        class_assignments[idx1] = i
        class_assignments[idx2] = i

    # 2. Iteratively Add Points to Bins:
    unassigned_leaves = np.where(class_assignments == -1)[0]
    print(f'These are the unassigned leaves: {unassigned_leaves}')
    print(f'This is the length of unassigned leaves: {len(unassigned_leaves)}')

    for leaf_idx in unassigned_leaves:
        mean_distances = [np.mean(distance_matrix[leaf_idx, class_assignments == bin_num])
                          for bin_num in range(n_initial_bins)]
        
        # Find best bin (highest mean distance above threshold)
        best_bin = np.argmax(mean_distances)
        if mean_distances[best_bin] >= distance_threshold:
            # Check individual distances within the best bin
            if all(distance_matrix[leaf_idx, class_assignments == best_bin] >= distance_threshold):
                class_assignments[leaf_idx] = best_bin
            else:
                # Try next best bin until a suitable bin is found or none exist
                sorted_bins = np.argsort(mean_distances)[::-1]  # Descending order
                for bin_idx in sorted_bins[1:]:  # Skip the already checked best bin
                    if all(distance_matrix[leaf_idx, class_assignments == bin_idx] >= distance_threshold):
                        class_assignments[leaf_idx] = bin_idx
                        break

    return class_assignments


In [22]:
import numpy as np

def phylogenetic_clustering(distance_matrix, n_initial_bins, distance_threshold):
    
    """
    Clusters terminal leaves of a phylogenetic tree into bins based on distance,
    prioritizing phylogenetic relationships.

    Args:
        distance_matrix: A square numpy array representing pairwise distances.
        n_initial_bins: The number of initial bins to create with the most distant points.
        distance_threshold: The minimum allowable distance for bin assignment.

    Returns:
        A numpy array of class assignments (bin numbers) for each terminal leaf.
    """

    n_leaves = distance_matrix.shape[0]
    class_assignments = np.full(n_leaves, -1, dtype=int)  # Initialize as -1 (unassigned)

    # 1. Initialize Bins with Most Distant Points:
    initial_points = farthest_points(distance_matrix=distance_matrix , k=n_initial_bins)
    print(f'These are the intial points: {initial_points}')
    print(f'This is the length of the intial points: {len(initial_points)}')

    for i, idx in enumerate(initial_points):
        class_assignments[idx] = i

    # 2. Iteratively Add Points to Bins:
    unassigned_leaves = np.where(class_assignments == -1)[0]
    print(f'These are the unassigned leaves: {unassigned_leaves}')
    print(f'This is the length of unassigned leaves: {len(unassigned_leaves)}')

    for leaf_idx in unassigned_leaves:
        mean_distances = [np.mean(distance_matrix[leaf_idx, class_assignments == bin_num])
                          for bin_num in range(n_initial_bins)]
        
        # Find best bin (highest mean distance above threshold)
        best_bin = np.argmax(mean_distances)
        if mean_distances[best_bin] >= distance_threshold:
            # Check individual distances within the best bin
            if all(distance_matrix[leaf_idx, class_assignments == best_bin] >= distance_threshold):
                class_assignments[leaf_idx] = best_bin
            else:
                # Try next best bin until a suitable bin is found or none exist
                sorted_bins = np.argsort(mean_distances)[::-1]  # Descending order
                for bin_idx in sorted_bins[1:]:  # Skip the already checked best bin
                    if all(distance_matrix[leaf_idx, class_assignments == bin_idx] >= distance_threshold):
                        class_assignments[leaf_idx] = bin_idx
                        break

    return class_assignments


In [4]:
def percentile_threshold(distance_matrix, percentile=5):
    """Calculates a distance threshold based on a given percentile of pairwise distances."""
    distances = distance_matrix[np.triu_indices_from(distance_matrix, k=1)]  # Extract upper triangle
    return np.percentile(distances, percentile)


In [10]:
def mean_std_threshold(distance_matrix, std_dev_factor=1.5):
    """Calculates a distance threshold based on mean and standard deviation of pairwise distances."""
    distances = distance_matrix[np.triu_indices_from(distance_matrix, k=1)]
    mean_distance = np.mean(distances)
    std_dev = np.std(distances)
    return mean_distance - std_dev_factor * std_dev


In [23]:
def tree_diameter_threshold(distance_matrix, fraction=0.05):
    """
    Calculates a distance threshold based on a fraction of the tree's diameter.

    Args:
        distance_matrix: A square numpy array representing pairwise distances.
        fraction: The fraction of the tree diameter to use as the threshold. Default is 0.5.

    Returns:
        The calculated distance threshold.
    """
    tree_diameter = np.max(distance_matrix)  # The maximum distance in the matrix is the diameter
    threshold = fraction * tree_diameter
    return threshold

In [5]:
percentile_dist_threshold = percentile_threshold(dist_matrix)
percentile_dist_threshold

0.2500477849

In [12]:
mean_std_threshold = mean_std_threshold(dist_matrix)
mean_std_threshold

-0.1295097867664987

In [24]:
tree_diameter_threshold = tree_diameter_threshold(dist_matrix)
tree_diameter_threshold

0.30027326624500006

In [23]:
n_initial_bins = 10 
phylo_folds = phylogenetic_clustering(distance_matrix=dist_matrix, n_initial_bins=n_initial_bins, distance_threshold=percentile_dist_threshold)

These are the intial points: [174, 65, 157, 173, 177, 68, 175, 164, 302, 150]
This is the length of the intial points: 10
These are the unassigned leaves: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  66  67  69  70  71  72  73
  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90  91
  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107 108 109
 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127
 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145
 146 147 148 149 151 152 153 154 155 156 158 159 160 161 162 163 165 166
 167 168 169 170 171 172 176 178 179 180 181 182 183 184 185 186 187 188
 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206
 207 208 209 210 211 212 213 214 215 216 2

In [24]:
phylo_folds

array([ 2,  7,  3,  0,  6,  9,  4,  2,  8,  1,  5, -1,  6,  9,  7,  4, -1,
       -1, -1,  3,  1, -1, -1, -1, -1, -1,  0,  2, -1, -1, -1,  5, -1, -1,
       -1,  8, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  2,  7,
        3,  6,  0,  9,  4,  7,  6, -1, -1, -1, -1, -1, -1, -1,  1,  3,  0,
        5,  9,  0,  3,  4,  7,  6,  9,  2,  3,  0,  7,  1,  5,  4,  9,  6,
        0,  3,  7,  2,  9,  6,  4,  1,  5,  8, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1,  0,  3, -1, -1, -1,  7,  9,  6,  2,  4,  0,  3,  7,  1,
        5,  8,  3, -1, -1, -1,  9,  6,  2,  0,  4,  7,  1,  9,  0,  6,  5,
        1,  8,  4,  3,  7,  2,  0,  6,  9,  4,  3,  1,  7,  5,  9,  0,  6,
        9,  2,  8,  0,  2,  6,  4,  9,  5,  1,  3,  7,  7,  0,  6,  8,  9,
        4,  3,  2,  3,  0,  6,  5,  4,  6,  5,  6,  5,  0,  6,  8,  4,  5,
        3,  9,  6,  0,  2,  1,  4,  3,  6,  9,  0,  5,  7,  8,  6,  4,  2,
        0,  3,  1,  9,  5,  7,  6,  8, -1, -1, -1, -1,  0,  3,  4,  9,  2,
        6,  5,  1,  7,  9

In [25]:
unique_classes, class_counts = np.unique(phylo_folds, return_counts=True)
print(class_counts)

[85 37 21 22 31 29 24 37 25 19 32]


In [21]:
import numpy as np

def farthest_points(distance_matrix, k=10):
    n = distance_matrix.shape[0]

    # Calculate average distances (initially to all points)
    avg_distances = np.mean(distance_matrix, axis=1) 
    
    # Initialize cluster (here, we start with the point with max avg distance)
    point_list = [np.argmax(avg_distances)]
    
    while len(point_list) < k:
        # Find the point with the highest average distance to existing cluster members
        max_avg_dist = -1
        best_point = -1
        for i in range(n):
            if i in point_list:
                continue
            avg_dist_to_cluster = np.mean(distance_matrix[i, point_list])
            if avg_dist_to_cluster > max_avg_dist:
                max_avg_dist = avg_dist_to_cluster
                best_point = i
        
        # Add the best point to the cluster
        point_list.append(best_point)

    return point_list

In [20]:
# Example usage (assuming you have a distance_matrix)
farthest_10 = farthest_points(distance_matrix=dist_matrix, k=10)
print(farthest_10) 

[174, 65, 157, 173, 177, 68, 175, 164, 302, 150]


In [ ]:
import numpy as np
import pandas as pd
from Bio import Phylo
from sklearn.cluster import AgglomerativeClustering
from scipy.spatial.distance import squareform, pdist, cdist
import random
from sklearn.neighbors import kneighbors_graph
import matplotlib.pyplot as plt
import seaborn as sns



def block_k_fold_cv(tree_file, fold_list, linkage="complete"):
    """
    Performs phylogenetic block k-fold cross-validation.

    Args:
        tree_file (str): Path to the phylogenetic tree file (Newick format).
        fold_list (list): List of the number of folds to test.

    Returns:
        list: List of dictionaries, each containing fold assignments and distance metrics.
    """

    # Load phylogenetic tree
    tree = Phylo.read(tree_file, "newick")
    # Get tip names
    tip_names = [terminal.name for terminal in tree.get_terminals()]
    #dist_matrix = tree.distance_matrix().values

    #Create distance matrix from the tree
    dist_matrix = np.zeros((len(tree.get_terminals()), len(tree.get_terminals())))
    for i, terminal1 in enumerate(tree.get_terminals()):
        for j, terminal2 in enumerate(tree.get_terminals()):
            if i < j:  # Only calculate upper triangle to avoid redundancy
                dist_matrix[i, j] = tree.distance(terminal1, terminal2)
                dist_matrix[j, i] = dist_matrix[i, j]  # Mirror for symmetry
    
    cnct_matrix = create_connectivity_matrix(tree)
    connectivity = kneighbors_graph(cnct_matrix, n_neighbors=5, include_self=True, mode='connectivity')

    results = []

    for n_folds in fold_list:
        # Cluster-based fold assignment
        clustering = AgglomerativeClustering(n_clusters=n_folds, metric ='precomputed', linkage=linkage, compute_distances=True, connectivity=connectivity).fit(dist_matrix)        
        block_fold = clustering.labels_
        unique_labels, cluster_sizes = np.unique(block_fold, return_counts= True)
        print(len(clustering.children_))
        # Reorder the distance matrix based on the clustering result
        #reordered_dist_matrix = dist_matrix[clustering.children_ - 1, :][:, clustering.children_ - 1] 
        #plt.figure(figsize=(10, 7))
        #sns.heatmap(reordered_dist_matrix, cmap="YlGnBu", cbar_kws={'label': 'Distance'})
        #plt.title('Reordered Distance Matrix')
        #plt.show()

        # Create dictionary to map tip names to fold assignments
        tip_to_fold = dict(zip(tip_names, block_fold))
        
        cluster_dict = {}
        for index, class_label in enumerate(block_fold):
            if class_label not in cluster_dict:
                cluster_dict[class_label] = []
            cluster_dict[class_label].append(index)
        sorted_keys = sorted(cluster_dict.keys())
        cluster_dict = {key: cluster_dict[key] for key in sorted_keys}
        cluster_indices = []    
        for values in cluster_dict.values():
            cluster_indices.append(values)
        target_size = len(tip_to_fold.keys()) // n_folds
        print(f"Targe Size = {target_size}")
        for x, cluster in enumerate(cluster_sizes):
            print(f"Cluster {x}: {cluster} members")
        over_represented_clusters = [i for i, size in enumerate(cluster_sizes) if size > target_size]
        print(f"Over-Represented Clusters = {over_represented_clusters}")
        under_represented_clusters = [i for i, size in enumerate(cluster_sizes) if size < target_size]
        print(f"Under-Represented Clusters = {under_represented_clusters}")
        
        # Reassign members from over-represented clusters to under-represented clusters
        for over_cluster in over_represented_clusters:
            print(f"Target Over-Represented Cluster = {over_cluster}")
            print(f"Here are the elements of the Target Over-represented Cluster = {cluster_indices[over_cluster]}")
            #cluster_indices[over_cluster] = random.shuffle(cluster_indices[over_cluster])
            #print(f"Here are the elements of the Target Over-represented Cluster Now Shuffled = {cluster_indices[over_cluster]}")

            for index in cluster_indices[over_cluster]:
                print(f"Target index is {index} - which should be {tip_names[index]}")
                #distances to nearest point per cluster
                point_distances = []
                #Index for the closest points per cluster 
                point_indexes = []
                for under_cluster in under_represented_clusters:      
                    #start here - need to use cluster_indices         
                        print(f"Target Under-Represented Cluster = {over_cluster}")
                        print(f"Here are the elements of the Target Under-represented Cluster = {cluster_indices[under_cluster]}")
                        point_index = cluster_indices[under_cluster]
                        distances_to_index = [dist_matrix[index, ci] for ci in point_index]
                        dist_check = [tree.distance(tip_names[index], tip_names[ci]) for ci in point_index]
                        for i in range(len(distances_to_index)):
                            if dist_check[i] != distances_to_index[i]:
                                print(f"Distance {i} is {dist_check[i]} and {distances_to_index[i]}")
                                raise Exception("The indexes being compared do not align with the tree")
                        min_dist = min(distances_to_index)
                        point_distances.append(min_dist)
                        point_indexes.append(under_cluster)
                  
                # Return the minimum distance to the nearest centroid
                print(f"Here are is the list of point distances for {index}: {point_distances} \n And the list of corresponding clusters: {point_indexes}")
                min_distance = min(point_distances)
                print(f"Here is the minimum distance: {min_distance}")
                target = point_distances.index(min_distance)
                print(f"Here is the closest cluster: {point_indexes[target]}")
                closest_under_cluster = point_indexes[target]
                block_fold[index] = closest_under_cluster
                cluster_sizes[over_cluster] -= 1
                cluster_sizes[closest_under_cluster] += 1
                for x, cluster in enumerate(cluster_sizes):
                    print(f"Cluster {x}: {cluster} members")

                for under_rep in under_represented_clusters:
                    if cluster_sizes[under_rep] >= target_size:
                        under_represented_clusters.remove(under_rep)
                if len(under_represented_clusters) == 0:
                    break
                if cluster_sizes[over_cluster] == target_size:
                    break
            if len(under_represented_clusters) == 0:
                break
                     
        # Update dictionary to map tip names to fold assignments
        tip_to_fold = dict(zip(tip_names, block_fold))
            
        results.append({"n_folds": n_folds, "block_fold": block_fold, "tip_to_fold": tip_to_fold})
            
    return results, dist_matrix
